In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_time=pd.read_csv("ankur_100_sample_labels_time.csv")
print(df_time.info())
df_time.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   review_id             100 non-null    object 
 1   user_id               100 non-null    object 
 2   business_id           100 non-null    object 
 3   stars                 100 non-null    int64  
 4   useful                100 non-null    int64  
 5   funny                 100 non-null    int64  
 6   cool                  100 non-null    int64  
 7   text                  100 non-null    object 
 8   date                  100 non-null    object 
 9   text_no_stop          100 non-null    object 
 10  text_stemmed          100 non-null    object 
 11  review_length         100 non-null    int64  
 12  word_count            100 non-null    int64  
 13  aspect                100 non-null    int64  
 14  aspect_label          100 non-null    object 
 15  sentiment_polarity    10

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,text_no_stop,...,aspect,aspect_label,sentiment_polarity,reviews_per_business,Food Quality,Customer Service,Place,Menu & Price,Drinks,Time
0,Hp3pIEuuk6t1RnKPTyt3hQ,-8oKMxks2GNqF9P41eE9LA,JnKdPqmgppB0CY7EuXaSCQ,5,0,0,0,really enjoyed this place gouda fondue app wa...,4/28/2013 6:11,really enjoyed place gouda fondue app deliciou...,...,4,Specific Dishes and Restaurant,0.683333,137,1,1,0,0,0,0
1,mh1TcChvFaI6m4jvM5aR4Q,yrntNCr9bxPmKEU3ncjw8A,mi7JfdCRLfxzFG71oW0I1g,4,1,0,0,best coffeeespresso in town\n\nive been search...,1/26/2019 20:06,best coffeeespresso town ive searching good qu...,...,3,Menu and Pricing,0.416667,10,0,0,0,0,1,0
2,4XGBkfQXxpghKSbmGwtLoQ,BX2MDfo93cMOdsvAQ4-WDA,9ugpNKKhnYRa51qXoxUw_A,5,0,0,0,came here for dinner last night and the tacos ...,3/26/2016 1:45,came dinner last night tacos great lot meat sa...,...,2,Overall Experience,0.600000,237,1,0,0,0,0,0
3,YhaWL-ioYp7dRjIMd8Z-gw,lcgjvFXX-dtvGvwXJ-UqDg,bKbvmpGy_L96qVtNeR_HNA,5,1,0,0,i just wanted to thank himalayan hut for the s...,5/5/2020 2:44,wanted thank himalayan hut service theyve prov...,...,1,Service and Wait Time,0.029167,19,1,1,0,0,0,0
4,1Jo-xTnxSybbs1v8rcTW_A,hoegTWA35mCxzi59DkZNng,iUZEGx29miZObLd6_lt7Vg,5,0,0,0,love the huevos rancheros here i make it a poi...,1/28/2018 0:18,love huevos rancheros make point eat breakfast...,...,2,Overall Experience,0.400000,349,1,0,0,0,0,0


In [6]:
import pandas as pd

def count_values(column):
    counts = column.value_counts()
    count_0 = counts.get(0, 0)
    count_1 = counts.get(1, 0)
    count_minus_1 = counts.get(-1, 0)
    
    return pd.Series({'Count of 0': count_0, 'Count of 1': count_1, 'Count of -1': count_minus_1})

columns_to_count = ['Food Quality', ' Customer Service', 'Place', 'Menu & Price', 'Drinks', 'Time']

counts_result = df_time[columns_to_count].apply(count_values)

(counts_result)


,Food Quality,Customer Service,Place,Menu & Price,Drinks,Time
Count of 0,21,43,70,61,83,81
Count of 1,65,41,22,24,16,9
Count of -1,14,16,8,15,1,10


# Testing the perfomance of different labels on manually labeled 100 rows extracted through stratified sampling

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# Load the data
data = pd.read_csv('ankur_100_sample_labels_time.csv')  

# Select relevant columns
columns = ['text', 'Food Quality', ' Customer Service', 'Place', 'Menu & Price', 'Drinks', 'Time']
data = data[columns]

data[columns[1:]] = data[columns[1:]].fillna(0)

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Build a pipeline for text classification
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', MultinomialNB()) 
])

# Train the model for each aspect
for aspect in columns[1:]:
    print(f"\nTraining model for aspect: {aspect}")
    # Fit the model
    text_clf.fit(train_data['text'], train_data[aspect])

    # Predictions on the test set
    predictions = text_clf.predict(test_data['text'])

    # Evaluate the model
    accuracy = accuracy_score(test_data[aspect], predictions)
    report = classification_report(test_data[aspect], predictions)

    print(f"Accuracy for {aspect}: {accuracy:.2f}")
    print(f"Classification Report for {aspect}:\n{report}")


Training model for aspect: Food Quality
Accuracy for Food Quality: 0.70
Classification Report for Food Quality:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         1
           0       0.00      0.00      0.00         5
           1       0.70      1.00      0.82        14

    accuracy                           0.70        20
   macro avg       0.23      0.33      0.27        20
weighted avg       0.49      0.70      0.58        20


Training model for aspect:  Customer Service
Accuracy for  Customer Service: 0.50
Classification Report for  Customer Service:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         3
           0       0.67      0.40      0.50        10
           1       0.43      0.86      0.57         7

    accuracy                           0.50        20
   macro avg       0.37      0.42      0.36        20
weighted avg       0.48      0.50      0.45        2

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# Load the data
data = pd.read_csv('ankur_100_sample_labels_time.csv')  # Replace 'your_dataset.csv' with the actual file path

# Select relevant columns
columns = ['text', 'Food Quality', ' Customer Service', 'Place', 'Menu & Price', 'Drinks', 'Time']
data = data[columns]

# Fill NaN values in the target variables with 0
data[columns[1:]] = data[columns[1:]].fillna(0)

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Build a pipeline for text classification using Random Forest


text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', RandomForestClassifier())  # Random Forest Classifier
])



# Train the model for each aspect
for aspect in columns[1:]:
    print(f"\nTraining model for aspect: {aspect}")
    # Fit the model
    text_clf.fit(train_data['text'], train_data[aspect])

    # Predictions on the test set
    predictions = text_clf.predict(test_data['text'])

    # Evaluate the model
    accuracy = accuracy_score(test_data[aspect], predictions)
    report = classification_report(test_data[aspect], predictions)

    print(f"Accuracy for {aspect}: {accuracy:.2f}")
    print(f"Classification Report for {aspect}:\n{report}")



Training model for aspect: Food Quality
Accuracy for Food Quality: 0.70
Classification Report for Food Quality:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         1
           0       0.00      0.00      0.00         5
           1       0.70      1.00      0.82        14

    accuracy                           0.70        20
   macro avg       0.23      0.33      0.27        20
weighted avg       0.49      0.70      0.58        20


Training model for aspect:  Customer Service
Accuracy for  Customer Service: 0.70
Classification Report for  Customer Service:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         3
           0       0.80      0.80      0.80        10
           1       0.60      0.86      0.71         7

    accuracy                           0.70        20
   macro avg       0.47      0.55      0.50        20
weighted avg       0.61      0.70      0.65        2

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# Load the data
data = pd.read_csv('ankur_100_sample_labels_time.csv')  

# Select relevant columns
columns = ['text', 'Food Quality', ' Customer Service', 'Place', 'Menu & Price', 'Drinks', 'Time']
data = data[columns]

# Fill NaN values in the target variables with 0
data[columns[1:]] = data[columns[1:]].fillna(0)

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Build a pipeline for text classification using Random Forest
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', RandomForestClassifier(n_estimators=100, random_state=42))  # Random Forest Classifier
])

# Train the model for each aspect
for aspect in columns[1:]:
    print(f"\nTraining model for aspect: {aspect}")
    # Fit the model
    text_clf.fit(train_data['text'], train_data[aspect])

    # Predictions on the test set
    predictions = text_clf.predict(test_data['text'])

    # Evaluate the model
    accuracy = accuracy_score(test_data[aspect], predictions)
    report = classification_report(test_data[aspect], predictions)

    print(f"Accuracy for {aspect}: {accuracy:.2f}")
    print(f"Classification Report for {aspect}:\n{report}")



Training model for aspect: Food Quality
Accuracy for Food Quality: 0.70
Classification Report for Food Quality:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         1
           0       0.00      0.00      0.00         5
           1       0.70      1.00      0.82        14

    accuracy                           0.70        20
   macro avg       0.23      0.33      0.27        20
weighted avg       0.49      0.70      0.58        20


Training model for aspect:  Customer Service
Accuracy for  Customer Service: 0.65
Classification Report for  Customer Service:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         3
           0       0.73      0.80      0.76        10
           1       0.56      0.71      0.63         7

    accuracy                           0.65        20
   macro avg       0.43      0.50      0.46        20
weighted avg       0.56      0.65      0.60        2

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# Load the data
data = pd.read_csv('ankur_100_sample_labels_time.csv')  
# Select relevant columns
columns = ['text', 'Food Quality', ' Customer Service', 'Place', 'Menu & Price', 'Drinks', 'Time']
data = data[columns]

# Fill NaN values in the target variables with 0
data[columns[1:]] = data[columns[1:]].fillna(0)

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Create a dictionary to store results for each aspect
results = []

# Build a pipeline for text classification
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', MultinomialNB())  # You can replace this classifier with another (e.g., RandomForest, SVM, etc.)
])

# Train the model for each aspect
for aspect in columns[1:]:
    #print(f"\nTraining model for aspect: {aspect}")
    # Fit the model
    text_clf.fit(train_data['text'], train_data[aspect])

    # Predictions on the test set
    predictions = text_clf.predict(test_data['text'])

    # Evaluate the model
    accuracy = accuracy_score(test_data[aspect], predictions)
    report = classification_report(test_data[aspect], predictions, output_dict=True)

    # Store the results in a dictionary
    result = {
        'Aspect': aspect,
        'Model Used': 'Multinomial Naive Bayes',
        'Accuracy': accuracy,
        'Precision (positive)': report['1']['precision'],
        'Recall (positive)': report['1']['recall'],
        'F1-score (positive)': report['1']['f1-score'],
        'Precision (negative)': report['0']['precision'],
        'Recall (negative)': report['0']['recall'],
        'F1-score (negative)': report['0']['f1-score'],
    }

    results.append(result)

# Create a DataFrame from the results list
results_df_NB = pd.DataFrame(results)



In [18]:
results_df_NB

,Aspect,Model Used,Accuracy,Precision (positive),Recall (positive),F1-score (positive),Precision (negative),Recall (negative),F1-score (negative)
0,Food Quality,Multinomial Naive Bayes,0.70,0.700000,1.000000,0.823529,0.000000,0.0,0.000000
1,Customer Service,Multinomial Naive Bayes,0.50,0.428571,0.857143,0.571429,0.666667,0.4,0.500000
2,Place,Multinomial Naive Bayes,0.85,0.000000,0.000000,0.000000,0.850000,1.0,0.918919
3,Menu & Price,Multinomial Naive Bayes,0.60,0.000000,0.000000,0.000000,0.600000,1.0,0.750000
4,Drinks,Multinomial Naive Bayes,0.85,0.000000,0.000000,0.000000,0.850000,1.0,0.918919
5,Time,Multinomial Naive Bayes,0.80,0.000000,0.000000,0.000000,0.800000,1.0,0.888889


The provided result table shows the performance metrics of sentiment analysis models for different aspects of Yelp reviews. Here's an interpretation of the results:

Food Quality Aspect:

Model Used: Multinomial Naive Bayes
Accuracy: 0.70
Precision (positive): 0.70
Recall (positive): 1.00
F1-score (positive): 0.82
Precision (negative): 0.00
Recall (negative): 0.00
F1-score (negative): 0.00
Interpretation: The model achieved an accuracy of 70% for the Food Quality aspect. It has good precision, recall, and F1-score for the positive class, indicating that it performs well in identifying positive sentiments. However, it has low performance for the negative class, as indicated by the precision, recall, and F1-score of 0.00.

Customer Service Aspect:

Model Used: Multinomial Naive Bayes
Accuracy: 0.50
Precision (positive): 0.43
Recall (positive): 0.86
F1-score (positive): 0.57
Precision (negative): 0.67
Recall (negative): 0.40
F1-score (negative): 0.50
Interpretation: The model achieved an accuracy of 50% for the Customer Service aspect. It has moderate precision, recall, and F1-score for both positive and negative classes. The F1-score for the positive class is 0.57, indicating that the model is better at identifying positive sentiments.

Place Aspect:

Model Used: Multinomial Naive Bayes
Accuracy: 0.85
Precision (positive): 0.00
Recall (positive): 0.00
F1-score (positive): 0.00
Precision (negative): 0.85
Recall (negative): 1.00
F1-score (negative): 0.92
Interpretation: The model achieved an accuracy of 85% for the Place aspect. However, it has low performance for the positive class, as indicated by precision, recall, and F1-score of 0.00. It performs very well for the negative class, with a high F1-score of 0.92.

Menu & Price Aspect:

Model Used: Multinomial Naive Bayes
Accuracy: 0.60
Precision (positive): 0.00
Recall (positive): 0.00
F1-score (positive): 0.00
Precision (negative): 0.60
Recall (negative): 1.00
F1-score (negative): 0.75
Interpretation: The model achieved an accuracy of 60% for the Menu & Price aspect. Similar to the Place aspect, it has low performance for the positive class but performs well for the negative class with an F1-score of 0.75.

Drinks Aspect:

Model Used: Multinomial Naive Bayes
Accuracy: 0.85
Precision (positive): 0.00
Recall (positive): 0.00
F1-score (positive): 0.00
Precision (negative): 0.85
Recall (negative): 1.00
F1-score (negative): 0.92
Interpretation: The model achieved an accuracy of 85% for the Drinks aspect, similar to the Place aspect. It has low performance for the positive class but performs very well for the negative class with an F1-score of 0.92.

Time Aspect:

Model Used: Multinomial Naive Bayes
Accuracy: 0.80
Precision (positive): 0.00
Recall (positive): 0.00
F1-score (positive): 0.00
Precision (negative): 0.80
Recall (negative): 1.00
F1-score (negative): 0.89
Interpretation: The model achieved an accuracy of 80% for the Time aspect. Similar to the Menu & Price and Drinks aspects, it has low performance for the positive class but performs well for the negative class with an F1-score of 0.89.

Overall, it appears that the Multinomial Naive Bayes model used for sentiment analysis has good performance for identifying positive sentiments but struggles to identify negative sentiments for most of the aspects. The models seem to have a bias toward predicting positive sentiments. Depending on your project's goals, you may need to consider different models or techniques to improve the performance, especially for negative sentiment detection.

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load your preprocessed data
df = pd.read_csv('ankur_100_sample_labels_time.csv')

# Define the list of aspects you want to analyze
aspects = ['Food Quality', ' Customer Service', 'Place', 'Menu & Price', 'Drinks', 'Time']

# Create a list to store results for each aspect
results_list = []

# Iterate through each aspect
for aspect in aspects:
    # Split the data into training and testing sets
    X = df['text_no_stop']  # Text features
    y = df[aspect]  # Aspect sentiment labels (0 for negative, 1 for positive)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create a TF-IDF vectorizer to convert text into numerical features
    tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)

    # Initialize and train the machine learning model (Logistic Regression in this case)
    model = LogisticRegression()
    model.fit(X_train_tfidf, y_train)

    # Make predictions on the test data
    y_pred = model.predict(X_test_tfidf)

    # Evaluate the model's performance
    accuracy = accuracy_score(y_test, y_pred)
    classification_rep = classification_report(y_test, y_pred, output_dict=True)

    # Store the results in a dictionary
    result_dict = {
        'Aspect': aspect,
        'Model Used': 'Logistic Regression',  # Specify the model used here
        'Accuracy': accuracy,
        'Precision (positive)': classification_rep['1']['precision'],
        'Recall (positive)': classification_rep['1']['recall'],
        'F1-score (positive)': classification_rep['1']['f1-score'],
        'Precision (negative)': classification_rep['0']['precision'],
        'Recall (negative)': classification_rep['0']['recall'],
        'F1-score (negative)': classification_rep['0']['f1-score'],
    }
    
    results_list.append(result_dict)

# Create a DataFrame from the results list
results_df_logistic = pd.DataFrame(results_list)



In [20]:
results_df_logistic

,Aspect,Model Used,Accuracy,Precision (positive),Recall (positive),F1-score (positive),Precision (negative),Recall (negative),F1-score (negative)
0,Food Quality,Logistic Regression,0.70,0.700000,1.000000,0.823529,0.000000,0.0,0.000000
1,Customer Service,Logistic Regression,0.65,0.545455,0.857143,0.666667,0.777778,0.7,0.736842
2,Place,Logistic Regression,0.85,0.000000,0.000000,0.000000,0.850000,1.0,0.918919
3,Menu & Price,Logistic Regression,0.60,0.000000,0.000000,0.000000,0.600000,1.0,0.750000
4,Drinks,Logistic Regression,0.85,0.000000,0.000000,0.000000,0.850000,1.0,0.918919
5,Time,Logistic Regression,0.80,0.000000,0.000000,0.000000,0.800000,1.0,0.888889


Food Quality Aspect:

Model Used: Logistic Regression
Accuracy: 0.70
Precision (positive): 0.70
Recall (positive): 1.00
F1-score (positive): 0.82
Precision (negative): 0.00
Recall (negative): 0.00
F1-score (negative): 0.00
Interpretation: The Logistic Regression model achieves similar performance as the Multinomial Naive Bayes model for the Food Quality aspect. It has good precision, recall, and F1-score for the positive class but low performance for the negative class.

Customer Service Aspect:

Model Used: Logistic Regression
Accuracy: 0.65
Precision (positive): 0.55
Recall (positive): 0.86
F1-score (positive): 0.67
Precision (negative): 0.78
Recall (negative): 0.70
F1-score (negative): 0.74
Interpretation: The Logistic Regression model performs slightly better than the Multinomial Naive Bayes model for the Customer Service aspect. It has moderate precision, recall, and F1-score for both positive and negative classes.

Place Aspect:

Model Used: Logistic Regression
Accuracy: 0.85
Precision (positive): 0.00
Recall (positive): 0.00
F1-score (positive): 0.00
Precision (negative): 0.85
Recall (negative): 1.00
F1-score (negative): 0.92
Interpretation: The Logistic Regression model achieves the same performance as the Multinomial Naive Bayes model for the Place aspect, with high accuracy and excellent performance for the negative class but poor performance for the positive class.

Menu & Price Aspect:

Model Used: Logistic Regression
Accuracy: 0.60
Precision (positive): 0.00
Recall (positive): 0.00
F1-score (positive): 0.00
Precision (negative): 0.60
Recall (negative): 1.00
F1-score (negative): 0.75
Interpretation: The Logistic Regression model performs similarly to the Multinomial Naive Bayes model for the Menu & Price aspect, with low performance for the positive class and better performance for the negative class.

Drinks Aspect:

Model Used: Logistic Regression
Accuracy: 0.85
Precision (positive): 0.00
Recall (positive): 0.00
F1-score (positive): 0.00
Precision (negative): 0.85
Recall (negative): 1.00
F1-score (negative): 0.92
Interpretation: The Logistic Regression model achieves the same high accuracy and performance as the Multinomial Naive Bayes model for the Drinks aspect, with poor performance for the positive class and excellent performance for the negative class.

Time Aspect:

Model Used: Logistic Regression
Accuracy: 0.80
Precision (positive): 0.00
Recall (positive): 0.00
F1-score (positive): 0.00
Precision (negative): 0.80
Recall (negative): 1.00
F1-score (negative): 0.89
Interpretation: The Logistic Regression model performs similarly to the Multinomial Naive Bayes model for the Time aspect, with low performance for the positive class and good performance for the negative class.

Overall, the Logistic Regression model exhibits similar patterns of performance to the Multinomial Naive Bayes model. It performs well in identifying positive sentiments but struggles to identify negative sentiments for most aspects. Like the previous model, it also seems to have a bias toward predicting positive sentiments. Depending on your project's goals, you may consider different models or techniques to improve negative sentiment detection.

In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# Load the data
data = pd.read_csv('ankur_100_sample_labels_time.csv')  # Replace 'your_dataset.csv' with the actual file path

# Select relevant columns
columns = ['text', 'Food Quality', ' Customer Service', 'Place', 'Menu & Price', 'Drinks', 'Time']
data = data[columns]

# Fill NaN values in the target variables with 0
data[columns[1:]] = data[columns[1:]].fillna(0)

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Build a pipeline for text classification using Random Forest
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', RandomForestClassifier(n_estimators=100, random_state=42))  # Random Forest Classifier
])

# Train the model for each aspect
for aspect in columns[1:]:
    print(f"\nTraining model for aspect: {aspect}")
    # Fit the model
    text_clf.fit(train_data['text'], train_data[aspect])

    # Predictions on the test set
    predictions = text_clf.predict(test_data['text'])

    # Evaluate the model
    accuracy = accuracy_score(test_data[aspect], predictions)
    report = classification_report(test_data[aspect], predictions)

    print(f"Accuracy for {aspect}: {accuracy:.2f}")
    print(f"Classification Report for {aspect}:\n{report}")



Training model for aspect: Food Quality
Accuracy for Food Quality: 0.70
Classification Report for Food Quality:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         1
           0       0.00      0.00      0.00         5
           1       0.70      1.00      0.82        14

    accuracy                           0.70        20
   macro avg       0.23      0.33      0.27        20
weighted avg       0.49      0.70      0.58        20


Training model for aspect:  Customer Service
Accuracy for  Customer Service: 0.65
Classification Report for  Customer Service:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         3
           0       0.73      0.80      0.76        10
           1       0.56      0.71      0.63         7

    accuracy                           0.65        20
   macro avg       0.43      0.50      0.46        20
weighted avg       0.56      0.65      0.60        2

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# Load the data
data = pd.read_csv('ankur_100_sample_labels_time.csv')  # Replace 'your_dataset.csv' with the actual file path

# Select relevant columns
columns = ['text', 'Food Quality', ' Customer Service', 'Place', 'Menu & Price', 'Drinks', 'Time']
data = data[columns]

# Fill NaN values in the target variables with 0
data[columns[1:]] = data[columns[1:]].fillna(0)

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Build a pipeline for text classification
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', MultinomialNB())  # You can replace this classifier with another (e.g., RandomForest, SVM, etc.)
])

# Train the model for each aspect
for aspect in columns[1:]:
    print(f"\nTraining model for aspect: {aspect}")
    # Fit the model
    text_clf.fit(train_data['text'], train_data[aspect])

    # Predictions on the test set
    predictions = text_clf.predict(test_data['text'])

    # Evaluate the model
    accuracy = accuracy_score(test_data[aspect], predictions)
    report = classification_report(test_data[aspect], predictions)

    print(f"Accuracy for {aspect}: {accuracy:.2f}")
    print(f"Classification Report for {aspect}:\n{report}")







Training model for aspect: Food Quality
Accuracy for Food Quality: 0.70
Classification Report for Food Quality:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         1
           0       0.00      0.00      0.00         5
           1       0.70      1.00      0.82        14

    accuracy                           0.70        20
   macro avg       0.23      0.33      0.27        20
weighted avg       0.49      0.70      0.58        20


Training model for aspect:  Customer Service
Accuracy for  Customer Service: 0.50
Classification Report for  Customer Service:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         3
           0       0.67      0.40      0.50        10
           1       0.43      0.86      0.57         7

    accuracy                           0.50        20
   macro avg       0.37      0.42      0.36        20
weighted avg       0.48      0.50      0.45        2

## checking Perfomance on individual aspects

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Load your preprocessed data
df = pd.read_csv('ankur_100_sample_labels_time.csv')

# Define the aspect you want to analyze
aspect = 'Food Quality'  # Replace with the aspect you are working on

# Split the data into training and testing sets
X = df['text_no_stop']  # Text features
y = df[aspect]  # Aspect sentiment labels (0 for negative, 1 for positive)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a TF-IDF vectorizer to convert text into numerical features
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize and train the machine learning model (Logistic Regression in this case)
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test_tfidf)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the results
print(f'Aspect: {aspect}')
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_rep)

Aspect: Food Quality
Accuracy: 0.70
Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00         1
           0       0.00      0.00      0.00         5
           1       0.70      1.00      0.82        14

    accuracy                           0.70        20
   macro avg       0.23      0.33      0.27        20
weighted avg       0.49      0.70      0.58        20



The model performs well in identifying positive sentiments for "Food Quality" with a high recall (1.00) and a reasonable F1-score (0.82).

However, the model does not perform well in identifying negative sentiments, as indicated by precision, recall, and F1-score, all of which are 0 for the negative class.
The overall accuracy of 0.70 suggests that the model's performance is driven by its ability to identify positive sentiments, but it lacks the ability to identify negative sentiments.

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load your preprocessed data
df = pd.read_csv('ankur_100_sample_labels_time.csv')

# Define the aspect you want to analyze
aspect = 'Food Quality'  # Replace with the aspect you are working on

# Split the data into training and testing sets
X = df['text_no_stop']  # Text features
y = df[aspect]  # Aspect sentiment labels (0 for negative, 1 for positive)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a TF-IDF vectorizer to convert text into numerical features
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize and train the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_tfidf, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test_tfidf)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the results
print(f'Aspect: {aspect}')
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_rep)


Aspect: Food Quality
Accuracy: 0.65
Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00         1
           0       0.00      0.00      0.00         5
           1       0.72      0.93      0.81        14

    accuracy                           0.65        20
   macro avg       0.24      0.31      0.27        20
weighted avg       0.51      0.65      0.57        20



In [89]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# Load your preprocessed data
df = pd.read_csv('ankur_100_sample_labels_time.csv')

# Define the aspect you want to analyze
aspect = 'Food Quality'  # Replace with the aspect you are working on

# Split the data into training and testing sets
X = df['text_no_stop']  # Text features
y = df[aspect]  # Aspect sentiment labels (0 for negative, 1 for positive)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a TF-IDF vectorizer to convert text into numerical features
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Apply SMOTE to oversample the minority class (1 for positive sentiment)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Initialize and train the machine learning model (Logistic Regression in this case)
model = LogisticRegression()
model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test data
y_pred = model.predict(X_test_tfidf)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the results
print(f'Aspect: {aspect}')
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_rep)


Aspect: Food Quality
Accuracy: 0.75
Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00         1
           0       1.00      0.20      0.33         5
           1       0.74      1.00      0.85        14

    accuracy                           0.75        20
   macro avg       0.58      0.40      0.39        20
weighted avg       0.77      0.75      0.68        20



In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# Load your preprocessed data
df = pd.read_csv('ankur_100_sample_labels_time.csv')

# Define the aspect you want to analyze
aspect = ' Customer Service'  # Replace with the aspect you are working on

# Split the data into training and testing sets
X = df['text_no_stop']  # Text features
y = df[aspect]  # Aspect sentiment labels (0 for negative, 1 for positive)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a TF-IDF vectorizer to convert text into numerical features
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Apply SMOTE to oversample the minority class (1 for positive sentiment)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Initialize and train the machine learning model (Logistic Regression in this case)
model = LogisticRegression()
model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test data
y_pred = model.predict(X_test_tfidf)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the results
print(f'Aspect: {aspect}')
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_rep)


Aspect:  Customer Service
Accuracy: 0.70
Classification Report:
               precision    recall  f1-score   support

          -1       1.00      0.33      0.50         3
           0       0.88      0.70      0.78        10
           1       0.55      0.86      0.67         7

    accuracy                           0.70        20
   macro avg       0.81      0.63      0.65        20
weighted avg       0.78      0.70      0.70        20



In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# Load your preprocessed data
df = pd.read_csv('ankur_100_sample_labels_time.csv')

# Define the aspect you want to analyze
aspect = 'Place'  # Replace with the aspect you are working on

# Split the data into training and testing sets
X = df['text_no_stop']  # Text features
y = df[aspect]  # Aspect sentiment labels (0 for negative, 1 for positive)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a TF-IDF vectorizer to convert text into numerical features
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Apply SMOTE to oversample the minority class (1 for positive sentiment)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Initialize and train the machine learning model (Logistic Regression in this case)
model = LogisticRegression()
model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test data
y_pred = model.predict(X_test_tfidf)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the results
print(f'Aspect: {aspect}')
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_rep)


Aspect: Place
Accuracy: 0.65
Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00         1
           0       0.81      0.76      0.79        17
           1       0.00      0.00      0.00         2

    accuracy                           0.65        20
   macro avg       0.27      0.25      0.26        20
weighted avg       0.69      0.65      0.67        20



In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# Load your preprocessed data
df = pd.read_csv('ankur_100_sample_labels_time.csv')

# Define the aspect you want to analyze
aspect = 'Menu & Price'  # Replace with the aspect you are working on

# Split the data into training and testing sets
X = df['text_no_stop']  # Text features
y = df[aspect]  # Aspect sentiment labels (0 for negative, 1 for positive)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a TF-IDF vectorizer to convert text into numerical features
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Apply SMOTE to oversample the minority class (1 for positive sentiment)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Initialize and train the machine learning model (Logistic Regression in this case)
model = LogisticRegression()
model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test data
y_pred = model.predict(X_test_tfidf)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the results
print(f'Aspect: {aspect}')
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_rep)


Aspect: Menu & Price
Accuracy: 0.70
Classification Report:
               precision    recall  f1-score   support

          -1       1.00      0.25      0.40         4
           0       0.69      0.92      0.79        12
           1       0.67      0.50      0.57         4

    accuracy                           0.70        20
   macro avg       0.78      0.56      0.59        20
weighted avg       0.75      0.70      0.67        20



In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# Load your preprocessed data
df = pd.read_csv('ankur_100_sample_labels_time.csv')

aspect = 'Time'  
# Split the data into training and testing sets
X = df['text_no_stop']  # Text features
y = df[aspect]  # Aspect sentiment labels (0 for negative, 1 for positive)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a TF-IDF vectorizer to convert text into numerical features
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Apply SMOTE to oversample the minority class (1 for positive sentiment)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Initialize and train the machine learning model (Logistic Regression in this case)
model = LogisticRegression()
model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test data
y_pred = model.predict(X_test_tfidf)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the results
print(f'Aspect: {aspect}')
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_rep)


Aspect: Time
Accuracy: 0.80
Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00         3
           0       0.80      1.00      0.89        16
           1       0.00      0.00      0.00         1

    accuracy                           0.80        20
   macro avg       0.27      0.33      0.30        20
weighted avg       0.64      0.80      0.71        20



In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# Load your preprocessed data
df = pd.read_csv('ankur_100_sample_labels_time.csv')

# Define the aspect you want to analyze
aspect = 'Food Quality'  # Replace with the aspect you are working on

# Split the data into training and testing sets
X = df['text_no_stop']  # Text features
y = df[aspect]  # Aspect sentiment labels (0 for negative, 1 for positive)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a TF-IDF vectorizer to convert text into numerical features
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Apply SMOTE to oversample the minority class (1 for positive sentiment)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Initialize and train the Random Forest classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test data
y_pred = model.predict(X_test_tfidf)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the results
print(f'Aspect: {aspect}')
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_rep)


Aspect: Food Quality
Accuracy: 0.70
Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00         1
           0       1.00      0.20      0.33         5
           1       0.72      0.93      0.81        14

    accuracy                           0.70        20
   macro avg       0.57      0.38      0.38        20
weighted avg       0.76      0.70      0.65        20

